In [42]:
from dotenv import load_dotenv

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI


import os
import json
from pprint import pprint

import chromadb
from chromadb.config import Settings

import pandas as pd

## Lade Environment-Variablen

In [2]:
load_dotenv()

DATA_LOCATION = os.environ.get("DATA_LOCATION")

## Helferfunktionen

In [3]:
def get_dbs():
    """Erzeugt ChromaDB-Client und stellt Verbindung zu zwei Collections her"""
    persistent_client = chromadb.PersistentClient(
        path=f"{DATA_LOCATION}/chroma_db", settings=Settings(allow_reset=True)
    )  # should be created once and passed around

    teaser_db = Chroma(
        client=persistent_client,
        collection_name="story_teaser",
        embedding_function=OpenAIEmbeddings(),
    )

    statement_db = Chroma(
        client=persistent_client,
        collection_name="story_statement",
        embedding_function=OpenAIEmbeddings(),
    )

    return teaser_db, statement_db, persistent_client

In [5]:
def chroma_to_dataframe(chroma_results):
    """Kleiner Helfer, um aus den Ergebnissen von Anfragen gegen ChromaDB DataFrames mit aufgelösten Metadaten zu erzeugen"""
    df = pd.DataFrame()

    if isinstance(chroma_results, dict):
        df = pd.DataFrame(chroma_results)
    elif isinstance(chroma_results, list) and isinstance(chroma_results[0], Document):
        df = pd.DataFrame(map(lambda x: {"documents": x.page_content, "metadatas": x.metadata}, chroma_results))

    if not df.empty:
        metadata = pd.json_normalize(df.metadatas)
        df = pd.concat([df, metadata], axis=1).drop(columns="metadatas")

    return df

## Datenabfragen

### Verbindung zu Vektordatenbank herstellen

In [6]:
teaser_db, statement_db, _ = get_dbs()

In [7]:
# Kleiner Test

print("There are", teaser_db._collection.count(), "documents in 'teaser_db'")
print("There are", statement_db._collection.count(), "documents in 'statement_db'")

There are 2635 documents in 'teaser_db'
There are 16980 documents in 'statement_db'


### Einfache Ähnlichkeitssuche

In [8]:
res = statement_db.similarity_search("klimawandel insekten")

In [9]:
res

[Document(page_content='Klimawandel: Mehr Insekten führen zu Ernteausfällen', metadata={'expert_affiliation': 'Professorin für Naturschutz und Landschaftsökologie, Albert-Ludwigs-Universität Freiburg', 'expert_name': 'Prof. Dr. Alexandra-Maria Klein', 'publication_date': '2018-08-30', 'ressort': 'Klima & Umwelt', 'source': '/workspace/data/story_statement/18062_2.txt', 'start_index': 0, 'statement_no': '18062_2_0_statement', 'story_no': 18062, 'type': 'Research in Context', 'url': 'https://www.sciencemediacenter.de/alle-angebote/research-in-context/details/news/klimawandel-mehr-insekten-koennen-zu-groesseren-ernteausfaellen-fuehren'}),
 Document(page_content='Klimawandel: Mehr Insekten führen zu Ernteausfällen', metadata={'expert_affiliation': 'Leiter der Arbeitsgruppe Allgemeine Zoologie, Institut für Biologie, Martin-Luther-Universität Halle-Wittenberg und Mitglied Deutsches Zentrum für integrative Biodiversitätsforschung (iDiv) Halle-Jena-Leipzig', 'expert_name': 'Prof. Dr. Robert P

In [12]:
chroma_to_dataframe(res)

,documents,expert_affiliation,expert_name,publication_date,ressort,source,start_index,statement_no,story_no,type,url
0,Klimawandel: Mehr Insekten führen zu Ernteausf...,Professorin für Naturschutz und Landschaftsöko...,Prof. Dr. Alexandra-Maria Klein,2018-08-30,Klima & Umwelt,/workspace/data/story_statement/18062_2.txt,0,18062_2_0_statement,18062,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,Klimawandel: Mehr Insekten führen zu Ernteausf...,"Leiter der Arbeitsgruppe Allgemeine Zoologie, ...",Prof. Dr. Robert Paxton,2018-08-30,Klima & Umwelt,/workspace/data/story_statement/18062_1.txt,0,18062_1_0_statement,18062,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,"Verlust von Biodiversität bei 1,5 statt 2°C Te...",Leiter des Lehrstuhls für Tierökologie und Tro...,Prof. Dr. Ingolf Steffan-Dewenter,2018-05-17,Klima & Umwelt,/workspace/data/story_statement/18035_1.txt,1,18035_1_0_statement,18035,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,Klimawandel: Mehr Insekten führen zu Ernteausf...,"Leiter der Abteilung Agrarökologie, Department...",Prof. Dr. Teja Tscharntke,2018-08-30,Klima & Umwelt,/workspace/data/story_statement/18062_0.txt,0,18062_0_0_statement,18062,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


### Ähnlichkeitssuche mit Filtern

In [17]:
# UND-Filter nach story_no >= 23000 und type == "Research in Context"
# k = 10 (default = 4) für mehr Ergebnisse

res = teaser_db.similarity_search(
    query="klimawandel insekten",
    k=10,
    filter={"$and": [{"story_no": {"$gte": 23000}}, {"type": "Research in Context"}]},
)

In [18]:
res

[Document(page_content='Für die nun vorliegende Analyse ergänzten die Forschenden zum einen die Daten der Krefeld-Studie mit Probennahmen für spätere Jahre, allerdings in Regionen, die bei Hallmann nicht untersucht wurden. Angeregt wurden sie dabei durch die Beobachtung, dass die Biomasse der Insekten im Jahr 2022 größer war als der langjährige Trend erwarten ließ und in der Größenordnung der späten 1980er Jahre lagen. Dies kombinierten sie mit Wetterdaten, einschließlich Informationen über Temperaturen und Niederschläge während der Probenahme. Auch Wetteranomalien während der verschiedenen Lebensphasen der Insekten wurden berücksichtigt. Sie fanden, dass bis zu 75 Prozent der Schwankungen der in den Fallen gefundenen Biomasse erklärt werden können, wenn neben räumlichen und zeitlichen Variablen auch Wetterbedingungen und -anomalien einbezogen werden. Die Forschenden schließen daraus, dass Veränderungen des Wetters durch den Klimawandel auch in Deutschland schwerwiegende Folgen für Ins

In [19]:
chroma_to_dataframe(res)

,documents,publication_date,ressort,source,start_index,story_no,type,url
0,Für die nun vorliegende Analyse ergänzten die ...,2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,1358,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,"2017 erschien die Krefeld-Studie, für die 27 J...",2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,878,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,"Das SMC hat Expertinnen und Experten befragt, ...",2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,2355,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,Erklärt das Wetter den Insektenrückgang?\n\nDe...,2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,0,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
4,Metastudie sucht nach Ursachen für Insektenrüc...,2023-08-23,Klima & Umwelt,/workspace/data/story_teaser/23140.txt,0,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
5,Das Insektensterben ist gut belegt: Globale so...,2023-02-01,Klima & Umwelt,/workspace/data/story_teaser/23020.txt,1347,23020,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
6,Insektenvielfalt stärkt Holzertrag im Wald\n\n...,2023-04-26,Klima & Umwelt,/workspace/data/story_teaser/23064.txt,0,23064,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
7,Die Studie erscheint vor dem Hintergrund einer...,2023-04-26,Klima & Umwelt,/workspace/data/story_teaser/23064.txt,2018,23064,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
8,Der seit vielen Jahren intensiv beforschte und...,2023-08-23,Klima & Umwelt,/workspace/data/story_teaser/23140.txt,54,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
9,Insektenarten sind durch Naturschutzgebiete ni...,2023-02-01,Klima & Umwelt,/workspace/data/story_teaser/23020.txt,0,23020,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


### Alle Statements für ein Angebot

In [20]:
statements_one_story = statement_db.get(
    where={
        "url": "https://www.sciencemediacenter.de/alle-angebote/research-in-context/details/news/klimafreundliches-kuehlen-in-deutschland/"
    }
)

In [21]:
statements_one_story

{'ids': ['23075_0_0_statement',
  '23075_0_1_statement',
  '23075_0_2_statement',
  '23075_0_3_statement',
  '23075_0_4_statement',
  '23075_0_5_statement',
  '23075_0_6_statement',
  '23075_0_7_statement',
  '23075_0_8_statement',
  '23075_0_9_statement',
  '23075_0_10_statement',
  '23075_0_11_statement',
  '23075_1_0_statement',
  '23075_1_1_statement',
  '23075_1_2_statement',
  '23075_1_3_statement',
  '23075_1_4_statement',
  '23075_1_5_statement',
  '23075_2_0_statement',
  '23075_2_1_statement',
  '23075_2_2_statement',
  '23075_2_3_statement',
  '23075_2_4_statement'],
 'embeddings': None,
 'metadatas': [{'expert_affiliation': 'Leiter des Teams Klimaanpassung und Stadtökologie, Fachbereich Umwelt, Deutsches Institut für Urbanistik',
   'expert_name': 'Jens Hasse',
   'publication_date': '2023-05-22',
   'ressort': 'Klima & Umwelt',
   'source': '/workspace/data/story_statement/23075_0.txt',
   'start_index': 0,
   'statement_no': '23075_0_0_statement',
   'story_no': 23075,
  

.get() liefert ein anders als .similarity_search() ein ChromDB-spezifisches Format zurück und nicht die Abbildung auf den Document-Datentyp von LangChain

chrom_to_dataframe() kann aber auch mit diesem Format umgehen

In [26]:
statements_one_story_df = chroma_to_dataframe(statements_one_story)
statements_one_story_df.head(10)

,ids,embeddings,documents,expert_affiliation,expert_name,publication_date,ressort,source,start_index,statement_no,story_no,type,url
0,23075_0_0_statement,None,Klimafreundliches Kühlen in Deutschland\n\nDer...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,0,23075_0_0_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,23075_0_1_statement,None,"Tatsächlich sind aber die Möglichkeiten, heuti...",Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,726,23075_0_1_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,23075_0_2_statement,None,Gebäudeteilen und gleichzeitig zu zur Abkühlun...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,1701,23075_0_2_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,23075_0_3_statement,None,Beim Neubau und der Sanierung von Gebäuden kan...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,1781,23075_0_3_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
4,23075_0_4_statement,None,Auch im öffentlichen Raum – auf Straßen und Pl...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,2515,23075_0_4_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
5,23075_0_5_statement,None,"Dazu gehört auch eine Stadtplanung, die sowohl...",Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,3488,23075_0_5_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
6,23075_0_6_statement,None,Soziale Innovationen – also klimaangepasste Le...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,3901,23075_0_6_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
7,23075_0_7_statement,None,Effiziente und effektive technische Kühlungsan...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,4574,23075_0_7_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
8,23075_0_8_statement,None,Parallel dazu sollten zweitens öffentliche und...,Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,5353,23075_0_8_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
9,23075_0_9_statement,None,"Drittens sollten möglichst nur dort, wo konsta...",Leiter des Teams Klimaanpassung und Stadtökolo...,Jens Hasse,2023-05-22,Klima & Umwelt,/workspace/data/story_statement/23075_0.txt,6077,23075_0_9_statement,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


### Gruppierung von Statements in einem Angebot nach Expert:in

In [27]:
statements_by_expert = (
    statements_one_story_df.groupby(by=["expert_name", "expert_affiliation"])["documents"].agg(list).to_dict()
)

In [28]:
for (name, affiliation), statements in statements_by_expert.items():
    print(name, affiliation)
    print("-" * 20)
    print("\n".join(statements))
    print("\n" * 4)

Dr. André Kremonke Wissenschaftlicher Mitarbeiter am Lehrstuhl für Gebäudeenergietechnik und Wärmeversorgung, Institut für Energietechnik, Technische Universität Dresden
--------------------
Klimafreundliches Kühlen in Deutschland

Der vorliegende Bericht analysiert die aktuelle Faktenlage zu den vor allem in urbanen Gebieten zunehmend steigenden sommerlichen Wärmebelastungen und deren Auswirkungen sehr umfassend. Dabei werden sowohl Möglichkeiten zur Vermeidung von Wärmebelastungen aufgezeigt, als auch Technologien zur Kompensation von Kühllasten analysiert. Sehr zielführend ist dabei die Bewertung der CO2-Emissionen, die bei der Deckung der Kühlenergiebedarfe und im Lebenszyklus der dafür eingesetzten Technologien entstehen. Dies bildet einen Hebel, um den Kreislauf aus CO2-Emissionen und der dadurch hervorgerufenen Klimaerwärmung zu bremsen.
Der Bericht beleuchtet nicht nur die gesundheitlichen, sozialen, infrastrukturellen und ökonomischen Auswirkungen steigender Kühlbedarfe, sonde

### Teaser für ein Angebot rekonstruieren

In [29]:
teasers_one_story = teaser_db.get(
    where={
        "url": "https://www.sciencemediacenter.de/alle-angebote/research-in-context/details/news/klimafreundliches-kuehlen-in-deutschland/"
    }
)

In [31]:
teasers_one_story_df = chroma_to_dataframe(teasers_one_story)
teasers_one_story_df

,ids,embeddings,documents,publication_date,ressort,source,start_index,story_no,type,url
0,23075_0_teaser,None,Klimafreundliches Kühlen in Deutschland\n\nHäu...,2023-05-22,Klima & Umwelt,/workspace/data/story_teaser/23075.txt,0,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,23075_1_teaser,None,Die Autorinnen und Autoren präsentieren eine R...,2023-05-22,Klima & Umwelt,/workspace/data/story_teaser/23075.txt,784,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,23075_2_teaser,None,2017 machten Kühlsysteme 14 Prozent des Stromv...,2023-05-22,Klima & Umwelt,/workspace/data/story_teaser/23075.txt,1400,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,23075_3_teaser,None,"Das SMC hat Forschende gebeten, die Vorschläge...",2023-05-22,Klima & Umwelt,/workspace/data/story_teaser/23075.txt,2370,23075,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


In [32]:
teaser = teasers_one_story_df.documents.str.cat(sep="\n\n")
print(teaser)

Klimafreundliches Kühlen in Deutschland

Häuser oder Wohnungen mit Klimaanlagen zu kühlen, war in Deutschland lange kein Thema. Doch zwei Entwicklungen werden das ändern: Zum einen steigen durch den Klimawandel die Temperaturen gerade im Sommer stark, zum anderen wird die Bevölkerung in Deutschland immer älter. Gerade alte Menschen sind durch hohe Temperaturen besonders gefährdet. Klimaanlagen schützen vor den Risiken zunehmender Hitze, doch zugleich sind sie oft klimaschädlich: viele Kühlmittel sind sehr starke Treibhausgase – ein Teufelskreis. Ein Bericht des Büros für Technikfolgenabschätzung (TAB) beim Deutschen Bundestag beschreibt nun, wie sich in Deutschland klimafreundliche Kühlsysteme etablieren ließen, er wird am kommenden Montag erscheinen (siehe Primärquelle).

Die Autorinnen und Autoren präsentieren eine Reihe von Vorschlägen, wie Kühlsysteme klimafreundlicher gestaltet werden könnten – technisch etwa durch den Einsatz von natürlichen Kältemitteln mit geringem Treibhausgas

## Erste Chains mit LangChain

#### Einfache Chain mit String-Output

In [46]:
teaser_chain_prompt_template = """
You are an experienced science journalist. Please extract the 5 most important facts from the text delimited by ```

```
{TEXT}
```

Please answer in German.
"""

teaser_chain_prompt = PromptTemplate(template=teaser_chain_prompt_template, input_variables=["TEXT"])
teaser_chain = teaser_chain_prompt | ChatOpenAI(temperature=0, model="gpt-3.5-turbo") | StrOutputParser()

In [48]:
facts = teaser_chain.invoke({"TEXT": teaser})

In [49]:
print(facts)

Die fünf wichtigsten Fakten aus dem Text sind:

1. Durch den Klimawandel steigen die Temperaturen in Deutschland, insbesondere im Sommer, stark an. Dies stellt vor allem ältere Menschen vor erhöhte Risiken durch Hitze.

2. Klimaanlagen können vor den Gefahren der zunehmenden Hitze schützen, sind jedoch oft klimaschädlich, da viele Kühlmittel starke Treibhausgase sind.

3. Ein Bericht des Büros für Technikfolgenabschätzung (TAB) beim Deutschen Bundestag präsentiert Vorschläge, wie klimafreundliche Kühlsysteme in Deutschland etabliert werden könnten. Dazu gehören der Einsatz von natürlichen Kältemitteln mit geringem Treibhausgaspotenzial, Quartierskälte und Wärmepumpen.

4. Maßnahmen wie das Begrünen von Straßen und Gebäuden oder das Freihalten von Windschneisen für kühle Nachtwinde können den Bedarf an Kühlung reduzieren.

5. Kühlsysteme, einschließlich Klimaanlagen und Haushaltsgeräte wie Kühlschränke, machen 14 Prozent des Stromverbrauchs in Deutschland aus. Ein Viertel davon entfällt

#### Einfache Chain mit JSON-Output

In [53]:
teaser_chain_prompt_template = """
You are an experienced science journalist. 
Please extract the 5 most important facts from the text delimited by ``` and calculate a score from 1 to 5 for their importance.

```
{TEXT}
```

Output format is a list of JSON-Objects with keys 'fact', 'score'

Please answer in German.
"""

teaser_chain_prompt = PromptTemplate(template=teaser_chain_prompt_template, input_variables=["TEXT"])
teaser_chain = teaser_chain_prompt | ChatOpenAI(temperature=0, model="gpt-3.5-turbo") | StrOutputParser() | json.loads

In [54]:
facts_score = teaser_chain.invoke({"TEXT": teaser})

In [55]:
pprint(facts_score)

[{'fact': 'Klimaanlagen schützen vor den Risiken zunehmender Hitze, doch '
          'zugleich sind sie oft klimaschädlich: viele Kühlmittel sind sehr '
          'starke Treibhausgase – ein Teufelskreis.',
  'score': 5},
 {'fact': 'Die Autorinnen und Autoren präsentieren eine Reihe von Vorschlägen, '
          'wie Kühlsysteme klimafreundlicher gestaltet werden könnten – '
          'technisch etwa durch den Einsatz von natürlichen Kältemitteln mit '
          'geringem Treibhausgaspotenzial, Quartierskälte oder durch '
          'Wärmepumpen.',
  'score': 4},
 {'fact': 'Der Bedarf nach Kühlung lässt sich reduzieren, wenn Städte zum '
          'Beispiel Straßen und Gebäude begrünen oder Windschneiden für kühle '
          'Nachtwinde freihalten.',
  'score': 3},
 {'fact': 'Verhaltensänderungen – wie die Einführung einer Siesta nach dem '
          'Vorbild südlicher Länder oder Kälteräume – könnten helfen, mit '
          'zunehmender Hitze umzugehen, ohne mehr kühlen zu müssen.',
  